In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 5g pyspark-shell'
#os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [ ]:
spark.stop()

In [ ]:
#!yarn application -kill application_1667306389915_0105

In [ ]:
spark

In [ ]:
#spark.stop()

In [2]:
import os
import sys
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, FloatType
from pyspark.sql.functions import col,array_contains
from pyspark.ml.feature import CountVectorizer
import pyspark.sql.functions as f
from pyspark.sql.functions import col, window, regexp_replace
from pyspark.sql.window import Window
import re
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import VectorAssembler

In [ ]:
!hdfs dfs -ls /labs/slaba03/

In [ ]:
#laba03_train.csv
#purchase - факты покупки
#user_id - id пользователя
#item_id - id телепередачи

In [3]:
schema = StructType() \
      .add("user_id", IntegerType(), True) \
      .add("item_id", IntegerType(), True) \
      .add("purchase", IntegerType(), True)
      
df_user_train = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/labs/slaba03/laba03_train.csv")
df_user_train.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows



In [ ]:
#laba03_test.csv
#purchase - факты покупки
#user_id - id пользователя
#item_id - id телепередачи

In [4]:
schema = StructType() \
      .add("user_id", IntegerType(), True) \
      .add("item_id", IntegerType(), True) 
      
      
df_user_test = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/labs/slaba03/laba03_test.csv")

df_user_test.show(5)

+-------+-------+
|user_id|item_id|
+-------+-------+
|   1654|  94814|
|   1654|  93629|
|   1654|   9980|
|   1654|  95099|
|   1654|  11265|
+-------+-------+
only showing top 5 rows



In [ ]:
df_user_test.show(5)

In [ ]:
#laba03_items.csv
#item_id - Соответствует item_id в предыдущем файле.
#content_type - тип контента
#title - название передачи, текстовое поле.
#year - год выпуска передачи, число.
#genres - поле с жанрами передачи, разделёнными через запятую.

In [5]:
read_items_schema = StructType(fields=[StructField('item_id', IntegerType()), 
StructField('channel_id', IntegerType()),
StructField('datetime_availability_start', StringType()),
StructField('datetime_availability_stop', StringType()),
StructField('datetime_show_start', StringType()),
StructField('datetime_show_stop', StringType()),
StructField('content_type', IntegerType()),
StructField('title', StringType(), nullable=True),
StructField('year', FloatType(), nullable=True),
StructField('genres', StringType()),
StructField('region_id', IntegerType()),
]) 




df_items = spark.read.format("csv") \
      .option("header", True) \
      .option("sep", "\t")\
      .schema(read_items_schema) \
      .load("/labs/slaba03/laba03_items.csv")

df_items.show(5)

+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------+------+-------+---------+
|item_id|channel_id|datetime_availability_start|datetime_availability_stop|datetime_show_start|datetime_show_stop|content_type|               title|  year| genres|region_id|
+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------+------+-------+---------+
|  65667|      null|       1970-01-01T00:00:00Z|      2018-01-01T00:00:00Z|               null|              null|           1|на пробах только ...|2013.0|Эротика|     null|
|  65669|      null|       1970-01-01T00:00:00Z|      2018-01-01T00:00:00Z|               null|              null|           1|скуби ду: эротиче...|2011.0|Эротика|     null|
|  65668|      null|       1970-01-01T00:00:00Z|      2018-01-01T00:00:00Z|               null|              null|           1|гор

In [6]:
read_users_schema = StructType(fields=[StructField('user_id', IntegerType()), 
StructField('item_id', IntegerType()),
StructField('ts_start', IntegerType()),
StructField('ts_end', IntegerType()),
StructField('item_type', StringType()),
]) 



df_views_programmes = spark.read.format("csv") \
      .option("header", True) \
      .schema(read_users_schema) \
      .load("/labs/slaba03/laba03_views_programmes.csv")

df_views_programmes.show(5)

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
|      0|7101053|1491409931|1491411600|     live|
|      0|7101054|1491412481|1491451571|     live|
|      0|7101054|1491411640|1491412481|     live|
|      0|6184414|1486191290|1486191640|     live|
|    257|4436877|1490628499|1490630256|     live|
+-------+-------+----------+----------+---------+
only showing top 5 rows



In [ ]:
df_views_programmes.join(df_items, on="item_id").show(5)

In [ ]:
df_views_programmes.join(df_user_train, on="user_id").show(5)

In [ ]:
df_views_programmes.join(df_user_test, on="user_id").show(5)

In [ ]:
df_user_test.join(df_user_train, ["user_id","item_id"]).show(5)

In [ ]:
df_user_train.groupBy("purchase").count().collect()

In [ ]:
10904/(10904+5021720)

In [ ]:
df_user_train.filter("purchase = 1").show(10)

In [ ]:
df_user_train.filter(f.isnull("user_id")).count()

In [ ]:
df_user_train.filter(f.isnull("item_id")).count()

In [ ]:
df_user_train.filter(f.isnull("purchase")).count()

In [ ]:
#df_train_groupbyuser = df_user_train.groupBy("user_id")\
#             .agg(f.count("*").alias("user_items_cnt"), f.sum("purchase").alias("user_purchase_cnt"))
#             #.orderBy("user_purchase_cnt", ascending=False)
            
#df_train_groupbyuser.show(5)

In [ ]:
#df_user_train = df_user_train.join(df_train_groupbyuser, on="user_id", how="left") 
#df_user_train.show(5)

In [ ]:
#df_train_groupbyitem = df_user_train.groupBy("item_id")\
#             .agg(f.count("*").alias("item_users_cnt"), f.sum("purchase").alias("item_purchase_cnt"))
#             #.orderBy("user_purchase_cnt", ascending=False)
            
#df_train_groupbyitem.show(5)

In [ ]:
#df_user_train = df_user_train.join(df_train_groupbyitem, on="item_id", how="left") 
#df_user_train.show(5)

In [7]:
df_train_groupbyuser = df_user_train.groupBy("user_id")\
             .agg((f.sum("purchase")/f.count("*")).alias("user_avg_purchase")).cache()

df_train_groupbyuser.show(5)

+-------+--------------------+
|user_id|   user_avg_purchase|
+-------+--------------------+
| 754230|0.027575641516660282|
| 761341|3.875968992248062E-4|
| 776188|0.001152516327314637|
| 780033|7.757951900698216E-4|
| 798454|3.840245775729646...|
+-------+--------------------+
only showing top 5 rows



In [8]:
df_user_train_new = df_user_train.join(df_train_groupbyuser, on="user_id", how="left").cache()
df_user_train_new.show(5)

+-------+-------+--------+--------------------+
|user_id|item_id|purchase|   user_avg_purchase|
+-------+-------+--------+--------------------+
|   1654|  74107|       0|0.001947040498442...|
|   1654|  89249|       0|0.001947040498442...|
|   1654|  99982|       0|0.001947040498442...|
|   1654|  89901|       0|0.001947040498442...|
|   1654| 100504|       0|0.001947040498442...|
+-------+-------+--------+--------------------+
only showing top 5 rows



In [9]:
df_train_groupbyitem = df_user_train.groupBy("item_id")\
             .agg((f.sum("purchase")/f.count("*")).alias("item_avg_purchase")).cache()
             #.orderBy("user_purchase_cnt", ascending=False)
            
df_train_groupbyitem.show(5)

+-------+--------------------+
|item_id|   item_avg_purchase|
+-------+--------------------+
|   8638|0.001450326323422...|
|  95940|   7.097232079489E-4|
|  78113|0.001468428781204...|
|  74757|7.358351729212656E-4|
|  94819|7.163323782234957E-4|
+-------+--------------------+
only showing top 5 rows



In [10]:
df_user_train_new = df_user_train_new.join(df_train_groupbyitem, on="item_id", how="left").cache()
df_user_train_new.show(5)

+-------+-------+--------+--------------------+--------------------+
|item_id|user_id|purchase|   user_avg_purchase|   item_avg_purchase|
+-------+-------+--------+--------------------+--------------------+
|  74107|   1654|       0|0.001947040498442...|7.575757575757576E-4|
|  89249|   1654|       0|0.001947040498442...|0.001473839351510...|
|  99982|   1654|       0|0.001947040498442...|0.001480384900074...|
|  89901|   1654|       0|0.001947040498442...|7.336757153338225E-4|
| 100504|   1654|       0|0.001947040498442...|0.004964539007092199|
+-------+-------+--------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#df_items.count()

In [ ]:
#df_items.groupBy("channel_id").count().show(5)

In [ ]:
#df_items.filter("genres IS NULL").count()

In [ ]:
#df_items.filter("genres IS NOT NULL").count()

In [ ]:
#df_items.groupBy("content_type").count().show(5)

In [11]:
df_items2 = df_items.fillna('Фильмы', subset=['genres'])

In [12]:
df_items2 = df_items2.withColumn("genres_split",f.split("genres",","))\
                     .drop('channel_id', 'year', 'title', 'region_id', 'datetime_availability_start', 'datetime_availability_stop', 'datetime_show_start', 'datetime_show_stop').cache()
df_items2.filter(f.length("genres")>20).show(5,truncate=False)

+-------+------------+----------------------------------------------+----------------------------------------------------+
|item_id|content_type|genres                                        |genres_split                                        |
+-------+------------+----------------------------------------------+----------------------------------------------------+
|326    |1           |Ужасы,Триллеры,Драмы,Фантастика,Зарубежные    |[Ужасы, Триллеры, Драмы, Фантастика, Зарубежные]    |
|336    |1           |Ужасы,Комедии,Фантастика,Зарубежные           |[Ужасы, Комедии, Фантастика, Зарубежные]            |
|357    |1           |Комедии,Мелодрамы,Наши                        |[Комедии, Мелодрамы, Наши]                          |
|396    |1           |Детективы,Триллеры,Драмы,Фантастика,Зарубежные|[Детективы, Триллеры, Драмы, Фантастика, Зарубежные]|
|400    |1           |Фантастика,Боевики,Зарубежные                 |[Фантастика, Боевики, Зарубежные]                   |
+-------+-------

In [13]:
df_items4 = df_items2\
.select('item_id', 'content_type',
        'genres_split', f.explode('genres_split').alias("genres_explode")).cache()
df_items4.filter(f.length("genres")>20).show(5)

+-------+------------+--------------------+--------------+
|item_id|content_type|        genres_split|genres_explode|
+-------+------------+--------------------+--------------+
|    326|           1|[Ужасы, Триллеры,...|         Ужасы|
|    326|           1|[Ужасы, Триллеры,...|      Триллеры|
|    326|           1|[Ужасы, Триллеры,...|         Драмы|
|    326|           1|[Ужасы, Триллеры,...|    Фантастика|
|    326|           1|[Ужасы, Триллеры,...|    Зарубежные|
+-------+------------+--------------------+--------------+
only showing top 5 rows



In [ ]:
df_items4.filter("content_type=1").groupby("genres_explode").count().sort("genres_explode").show(100)

In [14]:
count_vectorizer = CountVectorizer(inputCol="genres_split", outputCol="genres_vector")

In [15]:
count_vectorizer_model = count_vectorizer.fit(df_items2)

In [16]:
df_items2 = count_vectorizer_model.transform(df_items2).cache()

In [ ]:
df_items2.filter(f.length("genres")>20).show(5,truncate=False)

In [17]:
df_items3 = df_items2.drop("genres","genres_split","content_type").cache()

In [18]:
df_user_train_new = df_user_train_new.join(df_items3, on="item_id", how="left").cache()
df_user_train_new.show(5)

+-------+-------+--------+--------------------+--------------------+--------------------+
|item_id|user_id|purchase|   user_avg_purchase|   item_avg_purchase|       genres_vector|
+-------+-------+--------+--------------------+--------------------+--------------------+
|  74107|   1654|       0|0.001947040498442...|7.575757575757576E-4|(84,[1,2],[1.0,1.0])|
|  89249|   1654|       0|0.001947040498442...|0.001473839351510...|(84,[1,53],[1.0,1...|
|  99982|   1654|       0|0.001947040498442...|0.001480384900074...|(84,[1,10,16,24,2...|
|  89901|   1654|       0|0.001947040498442...|7.336757153338225E-4|(84,[2,5,16,28],[...|
| 100504|   1654|       0|0.001947040498442...|0.004964539007092199|(84,[1,11,13],[1....|
+-------+-------+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#df_user_train = df_user_train.join(df_items3, on="item_id", how="left") 
#df_user_train.show(5)

In [ ]:
#df_user_train.groupBy("content_type").count().show(5)

In [ ]:
#df_user_train = df_user_train.drop("content_type")

In [ ]:
#df_user_train.show(5)

In [ ]:
#df_views_programmes.show(5)
# per user_id
# abs time watch
# avg time watch
# count live
# count prv

In [ ]:
#df_views_programmes.groupBy("item_type").count().show(5)

In [ ]:
#assembler = VectorAssembler(inputCols=["user_items_cnt","user_purchase_cnt","item_users_cnt","item_purchase_cnt","genres_vector"], outputCol="features")

In [19]:
assembler = VectorAssembler(inputCols=["user_avg_purchase","item_avg_purchase","genres_vector"], outputCol="features")

In [20]:
df_user_train_new=assembler.transform(df_user_train_new).cache()
df_user_train_new.show(5)

+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+
|item_id|user_id|purchase|   user_avg_purchase|   item_avg_purchase|       genres_vector|            features|
+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+
|  74107|   1654|       0|0.001947040498442...|7.575757575757576E-4|(84,[1,2],[1.0,1.0])|(86,[0,1,3,4],[0....|
|  89249|   1654|       0|0.001947040498442...|0.001473839351510...|(84,[1,53],[1.0,1...|(86,[0,1,3,55],[0...|
|  99982|   1654|       0|0.001947040498442...|0.001480384900074...|(84,[1,10,16,24,2...|(86,[0,1,3,12,18,...|
|  89901|   1654|       0|0.001947040498442...|7.336757153338225E-4|(84,[2,5,16,28],[...|(86,[0,1,4,7,18,3...|
| 100504|   1654|       0|0.001947040498442...|0.004964539007092199|(84,[1,11,13],[1....|(86,[0,1,3,13,15]...|
+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+
o

In [ ]:
#df_user_train=assembler.transform(df_user_train)

In [ ]:
#df_user_train.show(5)

In [ ]:
#!hdfs dfs -rm -r lab03_df_user_train

In [ ]:
#df_user_train.write.save("lab03_df_user_train")

In [ ]:
#train = df_user_train.sampleBy("purchase", fractions={0: 0.8, 1: 0.8}, seed=42).cache()
#test = df_user_train.join(train, on=["user_id","item_id"], how="leftanti").coalesce(10).cache()

In [21]:
train_1 = df_user_train_new.filter("purchase=1").cache()
train_1.count()

10904

In [22]:
train_0 = df_user_train_new.filter("purchase=0").limit(32712).cache()
train_0.count()

32712

In [23]:
train_all = train_1.union(train_0).cache()
train_all.count()

43616

In [24]:
train = train_all.sampleBy("purchase", fractions={0: 0.8, 1: 0.8}, seed=42).cache()
test = train_all.join(train, on=["user_id","item_id"], how="leftanti").coalesce(10).cache()

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="purchase", metricName='areaUnderROC')
evaluator_prob = BinaryClassificationEvaluator(rawPredictionCol="probability", labelCol="purchase", metricName='areaUnderROC')

In [26]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol="features", labelCol="purchase")

model = gbt.fit(train)

In [27]:
predictions = model.transform(test).cache()

predictions.show(4)

+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|user_id|item_id|purchase|   user_avg_purchase|   item_avg_purchase|       genres_vector|            features|       rawPrediction|         probability|prediction|
+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 747028|  10007|       1| 0.18617021276595744|7.220216606498195E-4|(84,[6,7,14],[1.0...|(86,[0,1,8,9,16],...|[-1.4578212187089...|[0.05138569644756...|       1.0|
| 753619|  99912|       1|0.001916443081640...|7.320644216691069E-4|(84,[2,5],[1.0,1.0])|(86,[0,1,4,7],[0....|[-1.1934076277266...|[0.08418364043870...|       1.0|
| 843682|  74371|       1|0.003109211037699184| 7.22543352601156E-4|(84,[1,6,12,13],[...|(86,[0,1,3,8,14,1...|[-1.1934076277266...|[0.08418364043870...|       1.0|
| 852684|  93666

In [ ]:
#model.save("lab03.model")

In [ ]:
##!hdfs dfs rm -r "lab03.model"

In [ ]:
#model.load("lab03.model")

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
evaluator_prob.evaluate(predictions)

In [28]:
df_user_test_prep = df_user_test.join(df_train_groupbyuser, on="user_id", how="left").cache()
df_user_test_prep.show(5)

+-------+-------+--------------------+
|user_id|item_id|   user_avg_purchase|
+-------+-------+--------------------+
|   1654|  94814|0.001947040498442...|
|   1654|  93629|0.001947040498442...|
|   1654|   9980|0.001947040498442...|
|   1654|  95099|0.001947040498442...|
|   1654|  11265|0.001947040498442...|
+-------+-------+--------------------+
only showing top 5 rows



In [29]:
df_user_test_prep = df_user_test_prep.join(df_train_groupbyitem, on="item_id", how="left").cache()

In [ ]:
df_user_test_prep.show(5)

In [31]:
df_user_test_prep = df_user_test_prep.join(df_items3, on="item_id", how="left").cache()
#df_user_test_prep = df_user_test_prep.drop("content_type")
df_user_test_prep.show(5)

+-------+-------+--------------------+--------------------+--------------------+
|item_id|user_id|   user_avg_purchase|   item_avg_purchase|       genres_vector|
+-------+-------+--------------------+--------------------+--------------------+
|   8389| 902134|0.002674818494459...|0.005979073243647235|(84,[7,15,20,23],...|
|   8389| 902188|0.006682389937106...|0.005979073243647235|(84,[7,15,20,23],...|
|   8389| 902209|0.001527300496372...|0.005979073243647235|(84,[7,15,20,23],...|
|   8389| 902317|0.001189060642092...|0.005979073243647235|(84,[7,15,20,23],...|
|   8389| 902416|0.003128666405944466|0.005979073243647235|(84,[7,15,20,23],...|
+-------+-------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#assembler = VectorAssembler(inputCols=["user_items_cnt","user_purchase_cnt","item_users_cnt","item_purchase_cnt","genres_vector"], outputCol="features")

In [32]:
df_user_test_prep=assembler.transform(df_user_test_prep).cache()

In [33]:
df_user_test_prep.show(5)

+-------+-------+--------------------+--------------------+--------------------+--------------------+
|item_id|user_id|   user_avg_purchase|   item_avg_purchase|       genres_vector|            features|
+-------+-------+--------------------+--------------------+--------------------+--------------------+
|   8389| 902134|0.002674818494459...|0.005979073243647235|(84,[7,15,20,23],...|(86,[0,1,9,17,22,...|
|   8389| 902188|0.006682389937106...|0.005979073243647235|(84,[7,15,20,23],...|(86,[0,1,9,17,22,...|
|   8389| 902209|0.001527300496372...|0.005979073243647235|(84,[7,15,20,23],...|(86,[0,1,9,17,22,...|
|   8389| 902317|0.001189060642092...|0.005979073243647235|(84,[7,15,20,23],...|(86,[0,1,9,17,22,...|
|   8389| 902416|0.003128666405944466|0.005979073243647235|(84,[7,15,20,23],...|(86,[0,1,9,17,22,...|
+-------+-------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [35]:
predictions_test = model.transform(df_user_test_prep).cache()

In [36]:
import pandas as pd
import numpy as np

In [37]:
pred_pd = predictions_test.select("user_id","item_id",f.col("probability").alias("purchase")).toPandas()

In [38]:
pred_pd = pred_pd.sort_values(by=['user_id','item_id'])

In [39]:
pred_pd['purchase'] = pred_pd['purchase'].apply(lambda x: x[1])

In [40]:
pred_pd.to_csv('lab03.csv', index=False)

In [ ]:
#predictions_test.createOrReplaceTempView("predictions_test_view")

In [ ]:
#predictions_test.write.mode("overwrite").save("lab03_predictions_test")

In [ ]:
#predictions_test.load("lab03_predictions_test")

In [ ]:
#predictions_test.select("user_id", "item_id", "probability").show(5, False, True)

In [ ]:
#predictions_test

In [ ]:
#def getV1(v):
#    return float(v[1])
#getProb1 = f.udf(lambda v: getV1(v), DoubleType())

In [ ]:
#predictions_test.select(f.col("user_id").cast('int'),f.col("item_id").cast('int'),getProb1(f.col("probability")).alias("purchase"))\
#                .orderBy("user_id","item_id").show(5)

In [ ]:
#result = predictions_test.select(f.col("user_id").cast('int'),f.col("item_id").cast('int'),getProb1(f.col("probability")).alias("purchase"))\
#                .orderBy("user_id","item_id")

In [ ]:
#result.show(5)

In [ ]:
#result.count()

In [ ]:
#df_user_test.count()

In [ ]:
#result.toPandas().to_csv('lab03.csv')

In [ ]:
#import pandas as pd
#import numpy as np
#df = pd.read_csv("lab03.csv")
#df.describe()
#pd.read_csv("lab03.csv").to_csv("lab03_index.csv")
#df

In [ ]:
#import pandas as pd
#import numpy as np

#pred_pd = predictions_test.select("user_id","item_id",f.col("probability").alias("purchase")).toPandas()
#pred_pd = pred_pd.sort_values(by=['user_id','item_id'])
#pred_pd['purchase'] = pred_pd['purchase'].apply(lambda x: x[1])
#pred_pd.to_csv('lab03.csv', index=False)

In [ ]:
#result_index = result.withColumn("", f.monotonically_increasing_id())

In [ ]:
#result_index = result_index.select("","user_id","item_id","purchase")

In [ ]:
#result_index.show(5)

In [ ]:
#result_index.count()

In [ ]:
#result_index.write.save("lab03_result_index")

In [ ]:
#!hdfs dfs -rm -R /user/elena.pavlyuk/lab03.csv

In [ ]:
#result_index.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("lab03.csv")

In [ ]:
#!hadoop fs -copyToLocal /user/elena.pavlyuk/lab03.csv /data/home/elena.pavlyuk

In [ ]:
#result_index.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("result03.csv")

In [ ]:
#!hadoop fs -copyToLocal /user/elena.pavlyuk/result03.csv /data/home/elena.pavlyuk

In [ ]:
#result.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("result03noId.csv")

In [ ]:
#!hadoop fs -copyToLocal /user/elena.pavlyuk/result03noId.csv /data/home/elena.pavlyuk